In [1]:
#東森新聞雲爬蟲練習
#練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
#學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [7]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

In [8]:
executable_path="D:\Anaconda3\Lib\site-packages\chromedriver.exe" 
browser = webdriver.Chrome(executable_path)
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

while True:
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # 等待一段時間讓網頁更新
    time.sleep(SCROLL_PAUSE_TIME)

    # 檢視目前網頁底端
    new_height = browser.execute_script("return document.body.scrollHeight")
    
    if new_height == last_height:
        print("到達頁面底端")
        break
    last_height = new_height
    print("網頁更新中...")

網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
到達頁面底端


In [9]:
#到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

In [10]:
#解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)

----------------------------------------------------------------------

[0] 財稅紓困2.0／紓困補助、醫護人員危險津貼擬免稅　財稅加碼一次看

▲財政部。（圖／資料照）
記者吳靜君／台北報導
新冠肺炎（COVID-19）疫情持續，衝擊國內內需市場，總統蔡英文出面宣布要再擴大紓困方案，總金額要拉高到1兆5百億元。財政部除了追加特別預算之外，也會盤點醫護人員、清潔人員等津貼與企業、計程車司機的紓困補助研議可以免稅。
特別預算　追加1500億元
總統蔡英文宣布，因應新冠肺炎，在第二階段的特別預算部分將達到1500億元規模，將會修改特別預算法案追加預算，政府要做產業跟庶民大眾後盾，加強加快加碼紓困方案，而追加的預算則由主計總處統籌分配。　
財政部長蘇建榮表示，之前600億元的特別預算，其中300億元是前幾年的歲計賸餘來因應，舉債300億元；第二階段追加1500億元也以舉債因應，整體舉債大約1800億元
醫護人員的津貼、企業紓困補助　可以免稅
依據紓困條例的規範，財政部正在盤點政府的津貼、補助是否可以免稅，最為外界所關注的就是醫護人員的危險津貼每日1萬元，是否可以免稅。
賦稅署署長李慶華表示，正在盤點的除了計程車司機的補貼、醫院的清潔工、以及勞動部鼓勵減班休息的勞工趁此時間進行訓練的補貼等。企業方面，政府給予航空公司、遊覽車業、大眾運輸業的紓困、補助等，都可以免稅。李慶華表示，盤點內容也包含是否可以用現行法規解釋，如果不行就要增修紓困條例。
----------------------------------------------------------------------

[1] 蘋果讓部分零售店員變身線上客服顧問　每人還配發一台iMac！

▲蘋果讓部分店員變成線上客服顧問。（圖／翻攝自macrumors）
記者陳心怡／外電報導
根據《macrumors》報導，由於新冠肺炎（COVID-19）仍在蔓延，蘋果最近通知了一些美國的零售門市員工，讓他們有機會在家裡臨時擔任線上顧問。
接受提議的零售店員工，可以從蘋果那邊獲得必要的設備，並透過電話或在線上聊天向客戶提供支援，並且能獲得少量現金獎勵。目前還不清楚這個在家辦公的職位會持續多久，有多少員工得到了這個機會，或者是否在其他國家也有這樣的安排。

文／洪綾襄
疫情嚴峻的當下，副總統當選人賴清德以身作則謹守中央防疫規範，盡量減少外出；日前出席《財訊》特別於露天場地舉辦的財富管理大獎活動，他還現場示範與得獎者「交叉側身、肘碰肘」的方式，來取代握手道賀的新形態防疫社交禮儀。
賴清德從2月訪美回台後就深居簡出，由於極少露面，近來網路上甚至出現簡體字謠言，散播他已得武漢肺炎過世的假訊息，這讓行事一向低調的賴清德笑著駁斥：「麥擱黑白講啦！我很好！台灣也很好！」
延伸閱讀：
2020《財訊》財富管理大獎暨財管調查結果公布 元大證券抱走7大獎 6成5民眾去年賺20％以上
----------------------------------------------------------------------

[11] 台指期連假前跌43點　分析師：1-2周持續震盪整理、市場恐慌情緒濃

▲台指期4月1日走勢。（圖／取自期交所）
記者余弦妙／台北報導
即將進入清明連假，台指期今（1）日開盤下跌7點，來到9588點，隨後震盪整理在平盤附近徘徊，盤中雖一度攻上9650點，漲55點，不過尾盤出現賣壓，拉回平盤以下來到9551點，終場以9552點作收，下跌43點，與現貨相較，逆價差111.63點。
展望後市，統一期貨投顧經理廖恩平分析，未來1-2周持續震盪整理，最快一周後才會止穩，屆時再看是否有反攻機會。
廖恩平表示，今天盤中美國傳出未來新增死亡人數會超出預期，因此導致美股電子盤一度下跌7百多點，而台指期則是因為連假前夕在收盤前也出現一股下壓，且從目前電子盤開盤來看也有出現明顯下壓格局，這也顯示目前市場上出現擔憂情緒。
廖恩平也說，連假前確實很難上漲，主要為靠近萬點壓力，再加上連假前會有一股調節壓力，實際上在連假期間不確定性上升，首先就是美國周五即將公布最新的非農就業數據，再來就是美國死亡案例會增加，以上原因都讓市場趨向保守觀望態度。
因此，廖恩平也說，目前台指期整個月線往下，因此推估在未來的一周至兩周都還會持續下跌，若是較樂觀的預估，就是會在月線及十年線間震盪整理，最快也要一周的時間整理，屆時才能看是否有機會止穩反攻。
----------------------------------------------------------------------

[12] 只花10元爽中2百萬發票大獎！新莊買小熊QQ軟糖、太保買統

▲統一發票。（圖／記者游芳男翻攝）
記者吳靜君／台北報導
財政部今（1）日下午公布109年1、2月期統一發票的中獎清冊，此次有10張消費金額不到100元，就幸運中了千萬大獎，其中有消費者只花12元買衛生紙就中大獎。
財政部表示，1、2月期中獎發票自4月6日起至到7月6日止，請民眾記得兌領，請民眾不要忽略了自己的權益。
檢視這10張千萬大獎幸運發票，其中有1張在台南市容富百貨開出，消費金額只有12元，財政部表示，消費者是買1包柔漾衛生紙就中獎。
其他還包括在新北市中和區的爭鮮，消費71元；在高雄楠梓區的多亨烘焙坊消費20元，都幸運中獎。
其他消費超過百元中獎的幸運兒，還包括一位是在台北市中正區，花了225元點Uber Eats，就成了千萬大獎得主。
Google paly開立的1690元買App發票，也中了千萬大獎。
----------------------------------------------------------------------

[22] 宏碁電競桌機稱霸泛歐市場　2月奪英國、北歐、奧地利市佔第一

▲宏碁電競桌機稱霸泛歐市場。（圖／記者姚惠茹攝）
記者姚惠茹／台北報導
宏碁（2353）今（1）日宣布，電競品牌席捲泛歐市場，根據最新市調資料顯示，宏碁電競桌機在今年2月份奪下英國、北歐及奧地利市佔冠軍寶座，並在瑞士以坐二望一之姿領先市場。
宏碁表示，旗下電競桌機2020年搶先布局泛歐市場，分別在英國以17.1%市佔、北歐26.6%市佔及奧地利31.8%市佔奪得銷售第一，並在瑞士以第2名的21.7%市佔持續搶攻市場，而宏碁在泛歐地區表現亮眼，主要歸功於緊密的通路商合作、因應市場的行銷活動以及成功的定價策略。
宏碁積極耕耘電競市場，除了提供軟硬體外，更贊助國際電競賽事，做為2月IEM電競賽事的官方贊助商，在活動首頁上宣傳Orion 3000和5000電競桌機，並成功將行銷宣傳導向銷售，創下佳績，更在北歐地區以搭載GTX166OS顯示卡的Nitro 50 N50-600奪下最佳銷售單品。
----------------------------------------------------------------------

[23] 5大電信5G進度一次看！中華電最快獲NCC通過取得准籌設許可

----------------------

▲美元的價值近幾周急劇上漲，導致全球美元短缺。（圖／路透）
記者王曉敏／綜合報導
為滿足全球對美元的需求，美國聯準會（Fed）再度出招！將為他國央行及貨幣當局建立一個臨時回購操作，以美元交易美國國債解美元荒。此機制將於4月6日啟動，暫定為期6個月。
近幾周來，隨著投資人紛紛湧向安全資產，美元的價值急劇上漲，導致全球美元短缺，並對新興市場更造成嚴重打擊，加劇人們對全球經濟的擔憂。 Fed目前已與14國央行訂規貨幣互換協議（FX Swap），而新的機制將與此互相配合使用。
該臨時機制將允許在紐約聯邦準備銀行有戶頭的外國貨幣機構能暫時將所持有的美國國債售予Fed以換取美元，並在回購協議所規定的到期日買回債券，以在此期間滿足暫時性的美元需求。Fed在新聞中補充道，此機制也支持當地以美元計價的市場，並穩固市場信心。
美國外交關係委員會經濟學家塞策（Brad Setser）表示，Fed此舉主要為那些已經擁有大量外匯存底的國家及地區（如台灣、香港及泰國，甚至是印度及中國大陸）的金融機構提供支援，但這對於土耳其、南非、印尼、黎巴嫩等缺少外匯存底的國家並無太大用處，這些國家仍不得不求助於IMF以渡過危機。
----------------------------------------------------------------------

[35] 台股收盤下跌44點！連假前夕量縮震盪　指數失守9700點關卡

▲ 台股。（資料照／記者湯興漢攝）
記者李瑞瑾／台北報導
清明連假即將到來，市場投資人觀望氣氛濃厚，台股今（1）日量縮震盪，在平盤附近徘徊，終場收在9663點、下跌44點或0.46%，成交量1173.72億元。
電子三雄中，台積電（2330）以271.5元作收、下跌0.91%；股王大立光（3008）收3760元、下跌1.83%；鴻海（2317）收在70元、小漲0.14%。
面對即將到來的連假，不確定性導致市場操作保守，盤面上題材各自表現，有部分題材如面板股、防疫抗疫、宅經濟概念股強勢走高。
類股指數漲跌互見，其中光電、資訊服務類股漲逾1%；汽車、水泥類股跌幅在1%以上，油電燃氣類股則受台塑化（6505）重挫影響，跌幅逾3%。
投資人也擔心連假結束後，下周一開市會「變盤」重挫，財政部長蘇建榮表示，國安基金從來沒有鬆懈，尤其是美國的動盪，是國安基金特別關注，國安基金已經取

▲財信傳媒董事長謝金河。（圖／ETtoday資料照）
記者謝仁傑／綜合報導
財信傳媒董事長謝金河今（1）日在臉書發文，指出面對百年一遇大瘟疫，全世界的政府都奮力展開自救行動，美國祭出2兆美元紓困方案、歐元區祭出7500億歐元自救、南韓50兆韓元，台灣也由行政院長蘇貞昌宣布1兆新台幣的紓困措施。
他指出這將是企業體質一次殘酷的檢驗，體質欠佳的企業可能倒下去，像負債比率偏高、手上現金少的公司，或是股價不到5元的企業，或是像裕隆突然端出244億虧損，面對著這個史上罕見大災難，這是重中之重的大考驗。
▲謝金河指出大量執行庫藏股的企業是值得關注的指標。（圖／謝金河臉書）
他按照執行張數、比率、詳細列表，點名這次大量執行庫藏股的企業。像集盛一次6萬張，占近一成股權；宏碁的23萬張、國產10萬張等，都是值得關注的指標。
----------------------------------------------------------------------

[47] 麗清科技2019年產能擴充！LED車燈營運衝刺「Q4虧轉盈」　全年EPS達0.18元

▲麗清科技公告2019年全年營運成果。（圖／pixabay）
記者楊絡懸／台北報導
LED車燈模組老字號「麗清科技」（3346）公告2019年全年營運成果。2019年全年合併營收為41.33億元，年減6.17%，麗清指出，為未來營運衝刺做準備而進行產能擴充，加上武漢及上海新廠建置初期投入相關成本費用增加及學習曲線成本，最後全年稅後淨利1302萬元，每股稅後盈餘（EPS）0.18元。
麗清指出，2019年Q4為傳統汽車產業旺季，單季合併營收為10.97億元，加上一次性相關費用皆已認列，因此單季營運虧轉盈，每股稅後盈餘（EPS）0.24元。2019年一次性認列包括部分存貨呆滯損失、非本業造成的CB提前贖回利息費用1500萬元、因增資員工認股產生酬勞成本500萬元及借款利息費用4000萬元等費用，還有新廠建置投入的成本費用。
為因應主要客戶新車款LED車燈模組訂單的需求，麗清表示，除了持續提升LED車燈研發設計技術能力，積極與客戶共同研發設計出高技術規格的LED車燈模組相關產品，也陸續於武漢及上海新廠擴建新廠與增加產線布建，同時建置專業研發團隊。
麗清說明，目前各生產基地在復工後均達90%以上，新廠產能也逐步開出，持續加強克服學習曲

▲受疫情衝擊，電影院業績慘跌8到9成。（示意圖／記者張一中攝）
記者林育綾／台北報導
為防疫新冠肺炎（COVID-19）蔓延，中央疫情指揮中心指揮官陳時中日前建議戲院單場「不要賣超過100張票」，國民黨立委林奕華今（1）日提到，陳時中應該沒時間看電影，事實上戲院近來業績慘跌8到9成，不少業者透露別說是1場百張，「1天都可能賣不到百張」，沈重租金也是問題。文化部長鄭麗君回應，已向行政院提出討論，針對有重大衝擊者有進一步紓困方案，預計在4月中提出「藝文紓困2.0」。
國民黨立委林奕華今（1）日在立法院教育及文化委員會質詢文化部長鄭麗君，詢問520即將來臨，鄭部長是否已被徵詢人事高升？並表示雖然樂見她高升，但是在目前藝文產業受衝擊的時期，很多人需要她。而鄭麗君回應，目前各部會每天都全力做好防疫工作，自己也每天想著協助藝文紓困，沒有聽聞此事。
針對中央疫情指揮中心指揮官陳時中日前建議戲院單場「不要賣超過100張票」，林奕華表示，「陳時中應該是沒時間看電影」，事實上戲院近來業績慘跌8到9成，不少業者透露別說是1場百張，「1天都可能賣不到百張」，甚至有6成以上戲院盼政府直接「下令停業」。
林奕華還提到，有戲院每月租金超過百萬，而首波藝文紓困補助針對事業最高只有250萬，租金衝擊是明顯問題，目前有相關局處針對公有場域提供租金減半，但私人不可能如此，有無可能由政府來補貼一半租金，列入將來的紓困預算？
▲文化部長鄭麗君表示，針對第二波的藝文紓困補助，目前正跟行政院討論中。（圖／記者屠惠剛攝）
鄭麗君回應，知道戲院受衝擊的情況，日前電影院業績較從前不到5成，如今不到2成，已經向行政院提出，而針對第二波的藝文紓困補助，目前正跟行政院討論中，只是現在無法逐一回答有哪些細項，不過都是以解燃眉之急為主，無論戲院是否停業也都會全力協助紓困，針對有重大衝擊者，將有專案請求，有進一步的紓困方案，也就是紓困會再提升，預計4月中提出「藝文紓困2.0方案」。
林奕華也叮嚀，期望不要有差別待遇。鄭麗君也說，過去SARS的經驗對藝文產業的協助有限，不足以作為參考，但文化部已經向行政院提出請求，會盡力協助各藝文產業紓困。
----------------------------------------------------------------------

[58] 消費需求回溫！中國大陸市場重現活力　三

▲財政部長蘇建榮。（圖／記者林敬旻攝）
記者吳靜君／台北報導
民眾黨立委蔡壁如今（1）日仍問及國安基金問題，她提到美國總統川普示警，新冠肺炎（COVID-19）疫情造成感染和死亡人數升高，未來兩周美國非常的痛苦，國安基金是否在連假過後進場。財政部長蘇建榮回應，國安基金沒有鬆懈。
美股昨日再次收低，道瓊指數單季跌幅達到21%，創下33年來最大跌幅。台股今日開盤則上漲18點、以9726點開出。財政部今日到立法院財政委員會並且接受質詢，蔡壁如仍然問及國安基金的問題，並且問蘇建榮連假（清明連假）過後是否仍會繼續進場護盤。
蘇建榮表示，財政部國庫署密切注意國際競爭情勢，特別歐美股市發展，國安基金從來沒有鬆懈，尤其是美國的動盪，是國安基金特別關注。蘇建榮表示，國安基金已經取得委員會的授權，如果台股下周一（6日）的變化，就會依照情況採取行動。
----------------------------------------------------------------------

[67] 媲美香港半山豪宅　「曉陽明」頂端客層詢問度高

▲伴山豪宅因能享受清淨又能遠觀大景，是世界不少名流最愛的住所之一。（圖／欣巴巴提供）
消費中心／綜合報導
現代人愈來愈重視居住品質，貼近大自然的有氧環境，能讓緊繃的心靈放鬆下來，是「宜居住宅」的重要條件。台北市北投區的「曉陽明」地處奇岩丹鳳山，自然生態就在家門口，為海拔75公尺的伴山豪宅，除了能遠眺動人美景，車行1分鐘即達北投商圈，生活機能乃至醫療資源都相當完整，環境清幽且兼顧生活便利性。
離台北市中心車程只需30分鐘就能親近大自然的「曉陽明」，坐擁陽明山系綿延綠意，東鄰軍艦岩、西有關渡平原，加上南邊下方的奇岩重劃區，周邊被面積約436座大安森林公園的綠海圍繞，天天都可以呼吸純淨空氣；大範圍綠覆還帶來約千種動植物與昆蟲，附近的親山步道沿路能欣賞四季花卉、台灣藍鵲等繽紛生態，白天登頂可見遼闊風景，晚上則有城市繁華夜景。
▲▼位於北投的「曉陽明」，走出家門就有機會看到台灣藍鵲（上圖）或黑脈樺斑蝶（下圖），自然生態豐富。
▲▼「曉陽明」離軍艦岩步道相當近，不論白天或夜晚前往，都能欣賞到不同的美景。
「曉陽明」另一項宜居優勢在其低密度生活環境。近3千坪的大基地上只有5棟類別墅建築，其他規劃為社區內植林造景，棟距充足。太平洋房屋代銷/屈世偉協理表示，社

▲華南永昌證券爆發內控重大疏失，短短8個營業日虧掉34億元。（圖／鏡週刊提供，下同）
圖、文／鏡週刊
全球股災引發台股重挫，官股金控華南金也成了重災戶，旗下券商華南永昌證券上週爆發操作衍生性金融商品—指數型認售權證出現鉅額虧損，短短8個交易日就虧掉34億元。
 
上週二（3月24日）傍晚，華南金控旗下百分之百持股的子公司華南永昌證券（以下簡稱華南永昌）在股市觀測站上公告，由於股市從3月12日開始大跌，導致該公司權證交易避險不及，截至3月23日為止，已經造成34.13億元的鉅額虧損。
據股市觀測站公告的資料顯示，華南永昌過去7年獲利33.12億元，也就是說，從3月12日到23日短短8個交易日，華南永昌就虧掉7年的獲利，上週四（26日）華南金控董事長張雲鵬在立法院接受立委質詢時，坦承：「華南永昌是操作權證賣出選擇權，同時用期貨空單避險，目前虧損恐無法縮小。」更讓各界好奇，什麼樣的操作，會讓一家老牌券商中招。
本刊接獲爆料指出，早在3月20日，華南永昌權證大虧損的消息就已經在業內流傳，「華南永昌在9千多點到1萬1千點之間（去年2月到11月之間）發行了太多的指數型認售權證，他們的操作模式又幾乎不避險，賭這麼大一定會很慘！」業內人士透露。
台股重挫導致不少投資人財富縮水，理應做好避險的華南永昌卻因操作衍生性金融商品，成股災重災戶，還燒掉7年獲利，「難以置信！這一定是內控出問題。」一位券商高層對本刊直言。
「華南永昌因為發行太多台股指數型認售權證，可容忍的台股指數波動度約19％（只要台股加權指數每天波動在1.3％以內就會賺錢，一旦超過就會以等比級數出現虧損），沒想到3月12日以後，台股重挫導致波動度高達70％，他們想收回權證，投資人卻不願賣回給華南永昌，只要市場再繼續波動，虧損根本無法停止。」一位券商董事長對本刊透露。
華南金控旗下百分之百持股的子公司華南永昌證券（以下簡稱華南永昌）在股市觀測站上公告，由於股市從3月12日開始大跌，導致該公司權證交易避險不及，截至3月23日為止，已經造成34.13億元的鉅額虧損。
「華南永昌公告中所謂的避險不及，根本是推卸責任的說法。」知情人士表示，「像是凱基證券也有發認售權證，但他們在發行時，卻會用華南永昌發行的認售權證來當避險部位，防止台股波動過大，重點是看到同業大買，華南永昌還傻傻地擴大部位去賭，沒想到遇上了台股瘋狗浪，一次就畢業。」
「

▲響應政府政策，王道銀行推出「B型企業與社會企業融通貸款專案」，協助B型企業與社會企業度過難關。（圖／資料照）
記者紀佳妘／台北報導
新冠肺炎（COVID-19）疫情衝擊國內中小企業，王道銀行（O-Bank）今（31）日宣布，從4月1日起推出「B型企業與社會企業融通貸款專案」，提供額度最高600萬元、年利率1%的優惠貸款，並提供多項申貸手續費減免等優惠，以響應政府政策，並協助B型企業與社會企業共同度過難關。
王道銀行副董事長駱怡君表示，王道銀行推出「B型企業與社會企業優惠融通貸款專案」，是希望能以金融資源的支持，有效減緩疫情對B型企業和社會企業的影響，貸款額度最高可達新台幣600萬，以一般年利率1%以內的中小企業新冠肺炎融通貸款而言，額度算是相當高。
因應疫情對中小企業的衝擊，同時響應央行協助受肺炎疫情影響的中小企業融通貸款方案的政策，王道銀行推出「B型企業與社會企業融通貸款專案」，貸款額度最高可達新台幣600萬元，貸款優惠利率則以央行專案融通利率固定加0.75%，若以目前央行專案融通利率0.25%計算，貸款年利率僅1%。
至於該專案的對象範圍，王道銀指出，針對符合《中小企業認定標準》第2條，且同時為獲得國際B型企業認證的本國企業、或為於社團法人台灣公益團體自律聯盟「社會企業登錄平台」登錄的社會企業，提供徵提擔保品的優惠貸款。
「B型企業與社會企業融通貸款專案」從2020年4月1日起開始受理，至2021年3月27日止，浮動計息。此外，王道銀行更提供相關手續費全免，包括免帳戶管理費、免授信條件變更作業費等，若提前還款，也不須繳納違約金，以實際行動支持B型企業與社會企業的發展。
----------------------------------------------------------------------

[86] 隔離、檢疫每日1000元補償金　財政部核釋免所得稅

▲報綜所稅情形。（圖／記者林敬旻攝）
記者吳靜君／台北報導
財政部於今(31)日核釋，直轄市、縣市政府依「嚴重特殊傳染性肺炎隔離及檢疫期間防疫補償辦法」規定發給個人的防疫補償每人每日1000元，屬政府贈與，免納所得稅。
而政府給個人的防疫補償，是居家隔離、居家檢疫、集中隔離與集中檢疫，或者為照顧生活不能自理者的受隔離、檢疫，無法從事工作的家人，經衛生主管機關認定受隔離、檢疫者未違反隔離或檢疫